# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-04-13 18:18:28] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=37562, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=628888742, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=Fals

[2025-04-13 18:18:41 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-13 18:18:41 TP0] Init torch distributed begin.


[2025-04-13 18:18:41 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-13 18:18:41 TP0] Load weight begin. avail mem=59.09 GB


[2025-04-13 18:18:42 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-13 18:18:43 TP0] Using model weights format ['*.safetensors']
[2025-04-13 18:18:43 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2025-04-13 18:18:45 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-13 18:18:45] INFO:     Started server process [1639264]
[2025-04-13 18:18:45] INFO:     Waiting for application startup.
[2025-04-13 18:18:45] INFO:     Application startup complete.
[2025-04-13 18:18:45] INFO:     Uvicorn running on http://0.0.0.0:37562 (Press CTRL+C to quit)


[2025-04-13 18:18:46] INFO:     127.0.0.1:53776 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-13 18:18:46] INFO:     127.0.0.1:53784 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-13 18:18:46 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-13 18:18:49] INFO:     127.0.0.1:53798 - "POST /generate HTTP/1.1" 200 OK
[2025-04-13 18:18:49] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-04-13 18:18:51 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-13 18:18:51 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 6.35, #queue-req: 0, 


[2025-04-13 18:18:51 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 131.33, #queue-req: 0, 


[2025-04-13 18:18:52 TP0] Decode batch. #running-req: 1, #token: 121, token usage: 0.01, gen throughput (token/s): 127.15, #queue-req: 0, 
[2025-04-13 18:18:52] INFO:     127.0.0.1:34530 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "meta-llama/Llama-3.2-1B-Instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-04-13 18:18:52] INFO:     127.0.0.1:34546 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-04-13 18:18:52] INFO:     127.0.0.1:34562 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-04-13 18:18:52 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-13 18:18:53] INFO:     127.0.0.1:34572 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-04-13 18:18:53] INFO:     127.0.0.1:34580 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-04-13 18:18:53] INFO:     127.0.0.1:34592 - "POST /flush_cache HTTP/1.1" 200 OK
[2025-04-13 18:18:53 TP0] Cache flushed successfully!


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-04-13 18:18:53] Start update_weights. Load format=auto
[2025-04-13 18:18:53 TP0] Update engine weights online from disk begin. avail mem=38.52 GB
[2025-04-13 18:18:53 TP0] Using model weights format ['*.safetensors']


[2025-04-13 18:18:53 TP0] No model.safetensors.index.json found in remote.
[broadcast_pyobj] rank=0, device=cuda
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.70it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.70it/s]

[2025-04-13 18:18:54 TP0] Update weights end.
[2025-04-13 18:18:54 TP0] Cache flushed successfully!
[2025-04-13 18:18:54] INFO:     127.0.0.1:34604 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "meta-llama/Llama-3.2-1B-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "meta-llama/Llama-3.2-1B-wrong"
    " (repository not found)."
)

[2025-04-13 18:18:54] Start update_weights. Load format=auto
[2025-04-13 18:18:54 TP0] Update engine weights online from disk begin. avail mem=38.47 GB
[2025-04-13 18:18:54 TP0] Failed to get weights iterator: meta-llama/Llama-3.2-1B-wrong (repository not found).
[2025-04-13 18:18:54] INFO:     127.0.0.1:34606 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [10]:
terminate_process(server_process)

embedding_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-13 18:18:54] Child process unexpectedly failed with an exit code 9. pid=1639992
[2025-04-13 18:18:54] Child process unexpectedly failed with an exit code 9. pid=1639859


[2025-04-13 18:19:03] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-7B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-7B-instruct', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=36132, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=124807822, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=Fa

[2025-04-13 18:19:13 TP0] Downcasting torch.float32 to torch.float16.


[2025-04-13 18:19:13 TP0] Overlap scheduler is disabled for embedding models.
[2025-04-13 18:19:13 TP0] Downcasting torch.float32 to torch.float16.
[2025-04-13 18:19:13 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-13 18:19:13 TP0] Init torch distributed begin.


[2025-04-13 18:19:14 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-13 18:19:14 TP0] Load weight begin. avail mem=61.54 GB


...

[2025-04-13 18:19:27 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.55 GB, V size: 0.55 GB
[2025-04-13 18:19:27 TP0] Memory pool end. avail mem=45.74 GB


[2025-04-13 18:19:27 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-13 18:19:28] INFO:     Started server process [1641946]
[2025-04-13 18:19:28] INFO:     Waiting for application startup.
[2025-04-13 18:19:28] INFO:     Application startup complete.
[2025-04-13 18:19:28] INFO:     Uvicorn running on http://0.0.0.0:36132 (Press CTRL+C to quit)
[2025-04-13 18:19:28] INFO:     127.0.0.1:36088 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-13 18:19:29] INFO:     127.0.0.1:34438 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-13 18:19:29 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-13 18:19:30] INFO:     127.0.0.1:34442 - "POST /encode HTTP/1.1" 200 OK
[2025-04-13 18:19:30] The server is fired up and ready to roll!


In [11]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-04-13 18:19:33 TP0] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-13 18:19:33] INFO:     127.0.0.1:34450 - "POST /encode HTTP/1.1" 200 OK


In [12]:
terminate_process(embedding_process)

[2025-04-13 18:19:33] Child process unexpectedly failed with an exit code 9. pid=1642247
[2025-04-13 18:19:33] Child process unexpectedly failed with an exit code 9. pid=1642181


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [13]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-13 18:19:42] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, revision=None, host='0.0.0.0', port=39246, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=169197521, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=

[2025-04-13 18:19:51 TP0] Overlap scheduler is disabled for embedding models.
[2025-04-13 18:19:51 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-13 18:19:51 TP0] Init torch distributed begin.


[2025-04-13 18:19:52 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-13 18:19:52 TP0] Load weight begin. avail mem=59.43 GB


[2025-04-13 18:19:52 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-13 18:19:54 TP0] Using model weights format ['*.safetensors']


...

Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.18it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.79it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.52it/s]

[2025-04-13 18:19:57 TP0] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=30.36 GB, mem usage=29.06 GB.
[2025-04-13 18:19:57 TP0] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-04-13 18:19:57 TP0] Memory pool end. avail mem=27.57 GB


[2025-04-13 18:19:57 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-13 18:19:58] INFO:     Started server process [1643902]
[2025-04-13 18:19:58] INFO:     Waiting for application startup.
[2025-04-13 18:19:58] INFO:     Application startup complete.
[2025-04-13 18:19:58] INFO:     Uvicorn running on http://0.0.0.0:39246 (Press CTRL+C to quit)
[2025-04-13 18:19:58] INFO:     127.0.0.1:37710 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-13 18:19:59] INFO:     127.0.0.1:43908 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-13 18:19:59 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-13 18:20:00] INFO:     127.0.0.1:43918 - "POST /encode HTTP/1.1" 200 OK
[2025-04-13 18:20:00] The server is fired up and ready to roll!


In [14]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-04-13 18:20:04 TP0] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-13 18:20:04] INFO:     127.0.0.1:43930 - "POST /classify HTTP/1.1" 200 OK


In [15]:
terminate_process(reward_process)

[2025-04-13 18:20:04] Child process unexpectedly failed with an exit code 9. pid=1644238


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [16]:
expert_record_server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-04-13 18:20:14] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=30923, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=458859953, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=0, show_

[2025-04-13 18:20:24 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-13 18:20:24 TP0] Init torch distributed begin.


[2025-04-13 18:20:24 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-13 18:20:24 TP0] Load weight begin. avail mem=58.44 GB


[2025-04-13 18:20:24 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-13 18:20:25 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

[2025-04-13 18:20:31] INFO:     Started server process [1645812]
[2025-04-13 18:20:31] INFO:     Waiting for application startup.
[2025-04-13 18:20:31] INFO:     Application startup complete.
[2025-04-13 18:20:31] INFO:     Uvicorn running on http://0.0.0.0:30923 (Press CTRL+C to quit)


[2025-04-13 18:20:32] INFO:     127.0.0.1:35106 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-13 18:20:32] INFO:     127.0.0.1:35112 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-13 18:20:32 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-13 18:20:34 TP0] Using default MoE config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1b-gpu-23/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can tune the config with https://github.com/sgl-project/sglang/blob/main/benchmark/kernels/fused_moe_triton/tuning_fused_moe_triton.py.


[2025-04-13 18:20:35] INFO:     127.0.0.1:35120 - "POST /generate HTTP/1.1" 200 OK
[2025-04-13 18:20:35] The server is fired up and ready to roll!


In [17]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

import glob

output_file = glob.glob("expert_distribution_*.csv")[0]
with open(output_file, "r") as f:
    print_highlight("\n| Layer ID | Expert ID | Count |")
    print_highlight("|----------|-----------|--------|")
    next(f)
    for i, line in enumerate(f):
        if i < 9:
            layer_id, expert_id, count = line.strip().split(",")
            print_highlight(f"| {layer_id:8} | {expert_id:9} | {count:6} |")

[2025-04-13 18:20:37 TP0] Resetting expert distribution record...
[2025-04-13 18:20:37] INFO:     127.0.0.1:35134 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-04-13 18:20:37 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-13 18:20:37] INFO:     127.0.0.1:35142 - "POST /generate HTTP/1.1" 200 OK


[2025-04-13 18:20:37] INFO:     127.0.0.1:35150 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-04-13 18:20:37 TP0] Resetting expert distribution record...
[2025-04-13 18:20:37] INFO:     127.0.0.1:35160 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [18]:
terminate_process(expert_record_server_process)

[2025-04-13 18:20:37] Child process unexpectedly failed with an exit code 9. pid=1646161


## Skip Tokenizer and Detokenizer

SGLang Runtime also supports skip tokenizer and detokenizer. This is useful in cases like integrating with RLHF workflow.

In [19]:
tokenizer_free_server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --skip-tokenizer-init
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-13 18:20:47] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_path='meta-llama/Llama-3.2-1B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=True, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-1B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=39008, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=409284866, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=Fal

[2025-04-13 18:20:57 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-13 18:20:57 TP0] Init torch distributed begin.


[2025-04-13 18:20:58 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-13 18:20:58 TP0] Load weight begin. avail mem=78.58 GB
[2025-04-13 18:20:58 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-13 18:20:59 TP0] Using model weights format ['*.safetensors']
[2025-04-13 18:20:59 TP0] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.47it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.47it/s]

[2025-04-13 18:21:00 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=76.14 GB, mem usage=2.44 GB.
[2025-04-13 18:21:00 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.31 GB, V size: 0.31 GB
[2025-04-13 18:21:00 TP0] Memory pool end. avail mem=75.29 GB


[2025-04-13 18:21:00 TP0] 

CUDA Graph is DISABLED.
This will cause significant performance degradation.
CUDA Graph should almost never be disabled in most usage scenarios.
If you encounter OOM issues, please try setting --mem-fraction-static to a lower value (such as 0.8 or 0.7) instead of disabling CUDA Graph.

[2025-04-13 18:21:00 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072
[2025-04-13 18:21:00] INFO:     Started server process [1647740]
[2025-04-13 18:21:00] INFO:     Waiting for application startup.
[2025-04-13 18:21:00] INFO:     Application startup complete.
[2025-04-13 18:21:00] INFO:     Uvicorn running on http://127.0.0.1:39008 (Press CTRL+C to quit)


[2025-04-13 18:21:01] INFO:     127.0.0.1:58910 - "GET /v1/models HTTP/1.1" 200 OK
[2025-04-13 18:21:01] INFO:     127.0.0.1:58914 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-13 18:21:01 TP0] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-13 18:21:04] INFO:     127.0.0.1:58918 - "POST /generate HTTP/1.1" 200 OK
[2025-04-13 18:21:04] The server is fired up and ready to roll!


In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

input_text = "What is the capital of France?"

input_tokens = tokenizer.encode(input_text)
print_highlight(f"Input Text: {input_text}")
print_highlight(f"Tokenized Input: {input_tokens}")

response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "input_ids": input_tokens,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 256,
            "stop_token_ids": [tokenizer.eos_token_id],
        },
        "stream": False,
    },
)
output = response.json()
output_tokens = output["output_ids"]

output_text = tokenizer.decode(output_tokens, skip_special_tokens=False)
print_highlight(f"Tokenized Output: {output_tokens}")
print_highlight(f"Decoded Output: {output_text}")
print_highlight(f"Output Text: {output['meta_info']['finish_reason']}")

[2025-04-13 18:21:07 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-13 18:21:07 TP0] Decode batch. #running-req: 1, #token: 41, token usage: 0.00, gen throughput (token/s): 6.08, #queue-req: 0, 


[2025-04-13 18:21:07 TP0] Decode batch. #running-req: 1, #token: 81, token usage: 0.00, gen throughput (token/s): 201.87, #queue-req: 0, 
[2025-04-13 18:21:07] INFO:     127.0.0.1:58930 - "POST /generate HTTP/1.1" 200 OK


In [21]:
terminate_process(tokenizer_free_server_process)

[2025-04-13 18:21:07] Child process unexpectedly failed with an exit code 9. pid=1648227
[2025-04-13 18:21:07] Child process unexpectedly failed with an exit code 9. pid=1648096
